In [136]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

### 0. 测试上传数据
- 文件名: test_upload.csv
- 假设结果正态分布，用random数据。

In [137]:
test = pd.read_csv('season_1/test_0.csv',header=None);test.describe()


,0,2
count,2838.000000,2838.0
mean,33.500000,0.0
std,19.053729,0.0
min,1.000000,0.0
25%,17.000000,0.0
50%,33.500000,0.0
75%,50.000000,0.0
max,66.000000,0.0


In [19]:
test.describe()

,0,2
count,2838.000000,2838.0
mean,33.500000,0.0
std,19.053729,0.0
min,1.000000,0.0
25%,17.000000,0.0
50%,33.500000,0.0
75%,50.000000,0.0
max,66.000000,0.0


In [23]:
import random
test_list = []
for i in range(len(test)):
    test_list.append(abs(random.gauss(0,20)))
random_array = np.array(test_list)

In [35]:
type(random_array[1])

numpy.float64

In [25]:
random_forcast = pd.DataFrame(random_array)

In [31]:
test_result = pd.concat([test[0],test[1],random_forcast[0]],axis=1);test_result.columns=[0,1,2]

In [33]:
test_result.to_csv('test_upload.csv',header=False,index=False)

## 数据获取和清理

In [3]:
def output(day):
    order = pd.read_csv('season_1/training_data/order_data/order_data_{}'.format(day),delimiter='\t',header=None)
    order.columns = ["order_id", "driver_id","passenger_id","start_district_hash", "dest_district_hash", "Price","Time"]
    order["time"] = pd.to_datetime(order.Time,unit='s')
    order["time_id"] = order.time.apply(lambda x: x.hour*6 + int(x.minute/10)+1)
    # 地点信息
    district_data = pd.read_csv('season_1/training_data/cluster_map/cluster_map', delimiter='\t',header=None)
    district_data.columns = ["start_district_hash","district_id"]
    # 合并地区信息
    order_m = pd.merge(order,district_data,on="start_district_hash")
    # 简化表格，并计算gap
    std_df = order_m[["driver_id","time_id","district_id"]]
    # 整理 driver_id 信息，将 非NaN 都变为0，将NaN变为1
    mask = std_df.isnull()
    # convert the NaN data (gap) to 1 
    std_df['gap'] = np.where(mask.driver_id,1,0)
    # rid the hash data
    std_df =std_df.drop("driver_id",axis=1)
    # 
    std_output = std_df.groupby(["district_id","time_id"]).sum()
    
    std_output.to_csv('order_data_{}_gap.csv'.format(day),index=True)
    

In [78]:
for i in range(1,22):
    if i < 10:
        output("2016-01-0{}".format(i))
    else:
        output("2016-01-{}".format(i))

/Users/whale/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [135]:
# 尝试读取
df_test_output = pd.read_csv('./order_data_2016-01-{}_gap.csv'.format("15"))
df_test_output.iloc[0].argmax()

'gap'

### 尝试订单信息获得
- 具体步骤分解

In [142]:
order_1 = pd.read_csv('season_1/training_data/order_data/order_data_2016-01-01', delimiter='\t',header=None)
order_1.columns = ["order_id", "driver_id","passenger_id","start_district_hash", "dest_district_hash", "Price","Time"]

In [124]:
order_1.info()
# 可见 总体订单gap 为 (501287- 325577) / 501287 = 0.3505

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501287 entries, 0 to 501286
Data columns (total 7 columns):
order_id               501287 non-null object
driver_id              325577 non-null object
passenger_id           501287 non-null object
start_district_hash    501287 non-null object
dest_district_hash     501287 non-null object
Price                  501287 non-null float64
Time                   501287 non-null object
dtypes: float64(1), object(6)
memory usage: 26.8+ MB


### 订单信息数据整理
- 时间切片，获得 time_id 对应 144 个时区
- 地点切片，获得 district_id 对应的 66 个地区


In [125]:
# 如何把时间做切片？
order_1["time"] = pd.to_datetime(order_1.Time,unit='s');order_1.head()

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,Price,Time,time
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,2016-01-01 13:37:23
1,92c3ac9251cc9b5aab90b114a1e363be,c077e0297639edcb1df6189e8cda2c3d,191a180f0a262aff3267775c4fac8972,82cc4851f9e4faa4e54309f8bb73fd7c,b05379ac3f9b7d99370d443cfd5dcc28,2.0,2016-01-01 09:47:54,2016-01-01 09:47:54
2,abeefc3e2aec952468e2fd42a1649640,86dbc1b68de435957c61b5a523854b69,7029e813bb3de8cc73a8615e2785070c,fff4e8465d1e12621bc361276b6217cf,fff4e8465d1e12621bc361276b6217cf,9.0,2016-01-01 18:24:02,2016-01-01 18:24:02
3,cb31d0be64cda3cc66b46617bf49a05c,4fadfa6eeaa694742de036dddf02b0c4,21dc133ac68e4c07803d1c2f48988a83,4b7f6f4e2bf237b6cc58f57142bea5c0,4b7f6f4e2bf237b6cc58f57142bea5c0,11.0,2016-01-01 22:13:27,2016-01-01 22:13:27
4,139d492189ae5a933122c098f63252b3,NaN,26963cc76da2d8450d8f23fc357db987,fc34648599753c9e74ab238e9a4a07ad,87285a66236346350541b8815c5fae94,4.0,2016-01-01 17:00:06,2016-01-01 17:00:06


In [126]:
order_1["time_id"] = order_1.time.apply(lambda x: x.hour*6 + int(x.minute/10)+1)

In [128]:
# 地点信息
district_data = pd.read_csv('season_1/training_data/cluster_map/cluster_map', delimiter='\t',header=None)
district_data.columns = ["start_district_hash","district_id"];district_data.head()

,start_district_hash,district_id
0,90c5a34f06ac86aee0fd70e2adce7d8a,1
1,f2c8c4bb99e6377d21de71275afd6cd2,2
2,58c7a4888306d8ff3a641d1c0feccbe3,3
3,b26a240205c852804ff8758628c0a86a,4
4,4b9e4cf2fbdc8281b8a1f9f12b80ce4d,5


In [129]:
# 合并地理信息入数据
order_1_m = pd.merge(order_1,district_data,on="start_district_hash");order_1_m.head()

,order_id,driver_id,passenger_id,start_district_hash,dest_district_hash,Price,Time,time,time_id,district_id
0,97ebd0c6680f7c0535dbfdead6e51b4b,dd65fa250fca2833a3a8c16d2cf0457c,ed180d7daf639d936f1aeae4f7fb482f,4725c39a5e5f4c188d382da3910b3f3f,3e12208dd0be281c92a6ab57d9a6fb32,24.0,2016-01-01 13:37:23,2016-01-01 13:37:23,82,23
1,6fcae38baf2eb52e17273df41bf6fc6f,c586e492c4587c3aa15557a8eb41806d,c706cc35b05d84b224dec9d429c11de0,4725c39a5e5f4c188d382da3910b3f3f,4725c39a5e5f4c188d382da3910b3f3f,5.0,2016-01-01 14:28:07,2016-01-01 14:28:07,87,23
2,1b3b4b633578f88325973e084450af6f,7c630c98ba6679b30014807acc856d28,2a6c22435b8a3379a65176f9613458e4,4725c39a5e5f4c188d382da3910b3f3f,4725c39a5e5f4c188d382da3910b3f3f,7.0,2016-01-01 19:39:54,2016-01-01 19:39:54,118,23
3,bf9fcd57aa98208c11de945513e26cce,b29185649bd8d9c0fc04e44828b2a589,365e9ad6c8160b11ede6119b405dfbdd,4725c39a5e5f4c188d382da3910b3f3f,4725c39a5e5f4c188d382da3910b3f3f,9.0,2016-01-01 20:51:46,2016-01-01 20:51:46,126,23
4,9a13aebfdf99847fd502815271e45a01,be681893fd4ced097c0f50848d6dc013,637bb09709d9b0482244da8614fc64dd,4725c39a5e5f4c188d382da3910b3f3f,91690261186ae5bee8f83808ea1e4a01,43.3,2016-01-01 19:51:58,2016-01-01 19:51:58,120,23


In [130]:
# 简化表格，并计算gap
std_df = order_1_m[["driver_id","time_id","district_id"]]
# 整理 driver_id 信息，将 非NaN 都变为0，将NaN变为1
mask = std_df.isnull();mask
# convert the NaN data (gap) to 1 
std_df['gap'] = np.where(mask.driver_id,1,0);std_df.head()

/Users/whale/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,driver_id,time_id,district_id,gap
0,dd65fa250fca2833a3a8c16d2cf0457c,82,23,0
1,c586e492c4587c3aa15557a8eb41806d,87,23,0
2,7c630c98ba6679b30014807acc856d28,118,23,0
3,b29185649bd8d9c0fc04e44828b2a589,126,23,0
4,be681893fd4ced097c0f50848d6dc013,120,23,0


In [131]:
std_df =std_df.drop("driver_id",axis=1);std_df.head()

,time_id,district_id,gap
0,82,23,0
1,87,23,0
2,118,23,0
3,126,23,0
4,120,23,0


In [132]:
group0 = std_df.groupby(["district_id","time_id"])

In [133]:
std_output = group0.sum() 

In [76]:
# 获得order_1 的预测值了~
std_output.to_csv('order_1_gap.csv',index=True)

### 区域的地域属性数据获取

### 基准预测